In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import shutil

print(datetime.datetime.now())
print(os.getcwd())

2020-04-29 17:33:42.766234
/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/final_tables_to_use_in_model


In [2]:
os.listdir("/home/mysql/mysqloutfile/")

['sample557K_pred_pos_table_2_1.csv',
 'sample557K_pred_pos_table_2_2.csv',
 'new_sample557K_pred_pos_table_2_2.csv',
 'new_2_sample557K_pred_pos_table_2_2.csv',
 'new_3_sample557K_pred_pos_table_2_2.csv',
 'new_sample557K_pred_pos_table_2_1.csv',
 'new_4_sample557K_pred_pos_table_2_2.csv',
 'new_5_sample557K_pred_pos_table_2_2.csv']

In [68]:
df_POS_2_1="/home/mysql/mysqloutfile/new_sample557K_pred_pos_table_2_1.csv"
df_POS_2_2="/home/mysql/mysqloutfile/new_5_sample557K_pred_pos_table_2_2.csv"

shutil.copyfile(df_POS_2_1, "./"+os.path.basename(df_POS_2_1))
shutil.copyfile(df_POS_2_2, "./"+os.path.basename(df_POS_2_2))

'./new_5_sample557K_pred_pos_table_2_2.csv'

In [4]:
# store list of 20190901

df_store_list=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",dtype=str,sep="|")
df_store_list=df_store_list[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas']]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]



In [5]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)

df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [6]:
df_store_list=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_store_zip=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_store_zip.head(2)

,location_id,latitude_meas,longitude_meas,zip_cd,zip_type
0,1,39.913636,-82.914789,43227,P
1,1,39.913636,-82.914789,43207,P


In [7]:
df_store_zip_new=df_store_zip[pd.isnull(df_store_zip['zip_cd'])]
df_store_zip_existing=df_store_zip[pd.notnull(df_store_zip['zip_cd'])]

df_store_zip_new_no_loc=df_store_zip_new[df_store_zip_new['latitude_meas']==0]
df_store_zip_new_with_loc=df_store_zip_new[df_store_zip_new['latitude_meas']!=0]

store_list_later=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",
                             dtype=str,sep="|",usecols=['location_id','open_dt','latitude_meas','longitude_meas'])
store_list_later['latitude_meas']=store_list_later['latitude_meas'].astype(float)
store_list_later['longitude_meas']=store_list_later['longitude_meas'].astype(float)
df_store_zip_new_no_loc=store_list_later[store_list_later['location_id'].isin(df_store_zip_new_no_loc['location_id'].tolist())]

In [8]:
df_store_zip_new_with_loc=df_store_zip_new_with_loc[['location_id','latitude_meas','longitude_meas']]
df_store_zip_new_no_loc=df_store_zip_new_no_loc[['location_id','latitude_meas','longitude_meas']]
df_store_zip_new=df_store_zip_new_with_loc.append(df_store_zip_new_no_loc)
df_store_zip_new=df_store_zip_new.reset_index()
del df_store_zip_new['index']

In [9]:
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

df_all_new_zip=pd.DataFrame()
for i,row in df_store_zip_new.iterrows():
    store_coor=(row['latitude_meas'],row['longitude_meas'])
    store_num=row['location_id']
    list_store_zip=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_coor,v,unit="mi")
        if dist<=10:
            list_store_zip.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_zip,"zip_type":["zip_10"]*len(list_store_zip)},index=[store_num]*len(list_store_zip))
    df=df.reset_index().rename(columns={"index":"location_id"})
    df_all_new_zip=df_all_new_zip.append(df)
    
df_store_zip_new=pd.merge(df_store_zip_new,df_all_new_zip,on="location_id",how="left")
df_store_zip_new.shape

(202, 5)

In [10]:
df_store_zip=df_store_zip_existing.append(df_store_zip_new)
df_zip_type=df_store_zip[['zip_cd','zip_type']].drop_duplicates()
df_zip_type=df_zip_type.sort_values(['zip_cd','zip_type'])
print(df_zip_type['zip_type'].unique().tolist())
df_unique_zip_type=df_zip_type.drop_duplicates("zip_cd")

list_P_zips=df_zip_type[df_zip_type['zip_type']=="P"]['zip_cd'].tolist()
list_S_zips=df_zip_type[df_zip_type['zip_type']=="S"]['zip_cd'].tolist()
list_10_zips=df_zip_type[df_zip_type['zip_type']=="zip_10"]['zip_cd'].tolist()


['P', 'S', 'zip_10']


In [29]:
df_store_list=df_store_zip[['location_id','latitude_meas','longitude_meas']].drop_duplicates().reset_index()
del df_store_list['index']
df_store_list.head(2)

,location_id,latitude_meas,longitude_meas
0,1,39.913636,-82.914789
1,29,39.337172,-82.069765


In [54]:
i_counter

6360

In [59]:
# Add nearest BL store and the distance as of 20190901 store list

df_zips_with_BL_store=pd.DataFrame()
i_counter=0
for zip_cd,v in zip_centers.items():
    nearest_store=np.nan
    min_dist=np.inf
    for i,row in df_store_list.iterrows():
        store=row['location_id']
        store_loc=(row['latitude_meas'], row['longitude_meas'])
        dist=haversine(v,store_loc,unit="mi")
        if dist<=min_dist:
            min_dist=dist
            nearest_store=store
    df=pd.DataFrame({"nearest_BL_store":nearest_store,"nearest_BL_dist":min_dist},index=[zip_cd])
    df=df.reset_index().rename(columns={"index":"zip_cd"})
    df_zips_with_BL_store=df_zips_with_BL_store.append(df)
    i_counter+=1
    if i_counter%1000==1:
        print(datetime.datetime.now(),i_counter)
    

2020-04-29 18:36:12.964998 1
2020-04-29 18:41:22.775208 1001
2020-04-29 18:46:34.449834 2001
2020-04-29 18:51:49.983108 3001
2020-04-29 18:57:11.025382 4001
2020-04-29 19:02:24.957392 5001
2020-04-29 19:07:34.347900 6001
2020-04-29 19:12:45.513889 7001
2020-04-29 19:17:56.063749 8001
2020-04-29 19:23:04.611372 9001
2020-04-29 19:28:14.070818 10001
2020-04-29 19:33:21.855192 11001
2020-04-29 19:38:31.306442 12001
2020-04-29 19:43:42.339407 13001
2020-04-29 19:48:55.155357 14001
2020-04-29 19:54:06.218818 15001
2020-04-29 19:59:16.824102 16001
2020-04-29 20:04:26.364472 17001
2020-04-29 20:09:39.984890 18001
2020-04-29 20:14:49.649822 19001
2020-04-29 20:19:59.399061 20001
2020-04-29 20:25:07.179606 21001
2020-04-29 20:30:21.339739 22001
2020-04-29 20:35:51.709669 23001
2020-04-29 20:41:05.870098 24001
2020-04-29 20:46:13.138125 25001
2020-04-29 20:51:22.857730 26001
2020-04-29 20:56:33.064638 27001
2020-04-29 21:01:44.387751 28001
2020-04-29 21:06:58.026928 29001
2020-04-29 21:12:06.520

In [65]:
df_zips_with_BL_store.to_csv("./US_zip_to_BL_nearest_store_20190901_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# IV_list

In [81]:
high_date=datetime.date(2019, 8, 31)

In [82]:
df_1=pd.read_csv("./df_1_CRM.csv",dtype=str)
df_1=df_1[['customer_id_hashed','email_address_hash','sign_up_date','sign_up_channel','customer_zip_code']]
df_1['customer_zip_code']=df_1['customer_zip_code'].astype(str)
df_1['customer_zip_code']=df_1['customer_zip_code'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5)[:5])
df_1['sign_up_date']=pd.to_datetime(df_1['sign_up_date'],format="%Y-%m-%d").dt.date
df_1['weeks_since_sign_up']=df_1['sign_up_date'].apply(lambda x: int(np.ceil((high_date-x).days/7)))
df_1['P_zip']=np.where(df_1['customer_zip_code'].isin(list_P_zips),1,0)
df_1['S_zip']=np.where(df_1['customer_zip_code'].isin(list_S_zips),1,0)
df_1['else_10_zip']=np.where(df_1['customer_zip_code'].isin(list_10_zips),1,0)
# del df_1['customer_zip_code']
df_1['signed_online']=np.where(df_1['sign_up_channel']=="STORE",0,1)
del df_1['sign_up_channel']
print(df_1.shape,df_1['customer_id_hashed'].nunique())

df_1.head(2)

(577766, 9) 577766


,customer_id_hashed,email_address_hash,sign_up_date,customer_zip_code,weeks_since_sign_up,P_zip,S_zip,else_10_zip,signed_online
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,26454de1f315ee0c13c8c9033f372f50a3f02a16691849...,2018-09-22,93722,49,1,1,0,0
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,3ec23725736314cdfdd3fec2dc33de427da23104623312...,2010-09-05,04210,469,1,1,0,0


In [83]:
list_unsunsribe_ids=pd.read_csv("/home/jian/BigLots/unsubscribe/Unsub_JH_Final20191105Hashed.zip",
                         compression="zip",dtype=str,usecols=['hashed_customer_num'])['hashed_customer_num'].unique().tolist()

print(len(list_unsunsribe_ids))
df_1['email_unsub_19Oct']=np.where(df_1['customer_id_hashed'].isin(list_unsunsribe_ids),1,0)

2771375


In [87]:
df_zips_with_BL_store=df_zips_with_BL_store.rename(columns={"zip_cd":"customer_zip_code"})
df_1=pd.merge(df_1,df_zips_with_BL_store,on="customer_zip_code",how="left")
del df_1['customer_zip_code']

In [88]:
df_2_1=pd.read_csv("./new_sample557K_pred_pos_table_2_1.csv",na_values='\\N')
df_2_1=df_2_1.rename(columns={"id":"customer_id_hashed"})
print(df_2_1.shape,df_2_1['customer_id_hashed'].nunique())
df_2_1.head(2)

(577766, 62) 577766


,customer_id_hashed,week_diff,total_trans_since_registration,total_items,total_sales,trans_in_store,unique_stores,trans_online,department_minus_one_trans,department_108_trans,...,department_530_trans,department_540_trans,department_550_trans,department_560_trans,department_608_trans,department_610_trans,department_612_trans,department_615_trans,department_710_trans,department_800_trans
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,49,3,23,143.25,3,1,0,0,1,...,0,0,1,0,0,0,0,0,3,0
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,468,6,29,82.98,6,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df_2_2=pd.read_csv("./new_5_sample557K_pred_pos_table_2_2.csv",na_values='\\N')
df_2_2=df_2_2.rename(columns={"id":"customer_id_hashed"})
print(df_2_2.shape,df_2_2['customer_id_hashed'].nunique())
df_2_2.head(2)

(577766, 359) 577766


,customer_id_hashed,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,total_units_department_minus_1_1st_trans,total_units_department_108_1st_trans,total_units_department_109_1st_trans,total_units_department_110_1st_trans,total_units_department_111_1st_trans,...,total_units_department_530_recent_5_trans,total_units_department_540_recent_5_trans,total_units_department_550_recent_5_trans,total_units_department_560_recent_5_trans,total_units_department_608_recent_5_trans,total_units_department_610_recent_5_trans,total_units_department_612_recent_5_trans,total_units_department_615_recent_5_trans,total_units_department_710_recent_5_trans,total_units_department_800_recent_5_trans
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,1,58.7,7,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,2018-09-30,1,13.1,6,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
pd.options.display.max_columns=555
df_2_2[pd.isnull(df_2_2['week_recent_1_trans'])]

,customer_id_hashed,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,total_units_department_minus_1_1st_trans,total_units_department_108_1st_trans,total_units_department_109_1st_trans,total_units_department_110_1st_trans,total_units_department_111_1st_trans,total_units_department_114_1st_trans,total_units_department_115_1st_trans,total_units_department_120_1st_trans,total_units_department_130_1st_trans,total_units_department_140_1st_trans,total_units_department_150_1st_trans,total_units_department_160_1st_trans,total_units_department_170_1st_trans,total_units_department_210_1st_trans,total_units_department_230_1st_trans,total_units_department_250_1st_trans,total_units_department_270_1st_trans,total_units_department_310_1st_trans,total_units_department_320_1st_trans,total_units_department_330_1st_trans,total_units_department_340_1st_trans,total_units_department_350_1st_trans,total_units_department_351_1st_trans,total_units_department_352_1st_trans,total_units_department_353_1st_trans,total_units_department_354_1st_trans,total_units_department_355_1st_trans,total_units_department_360_1st_trans,total_units_department_361_1st_trans,total_units_department_362_1st_trans,total_units_department_363_1st_trans,total_units_department_364_1st_trans,total_units_department_365_1st_trans,total_units_department_366_1st_trans,total_units_department_367_1st_trans,total_units_department_370_1st_trans,total_units_department_410_1st_trans,total_units_department_425_1st_trans,total_units_department_470_1st_trans,total_units_department_480_1st_trans,total_units_department_510_1st_trans,total_units_department_520_1st_trans,total_units_department_521_1st_trans,total_units_department_526_1st_trans,total_units_department_530_1st_trans,total_units_department_540_1st_trans,total_units_department_550_1st_trans,total_units_department_560_1st_trans,total_units_department_608_1st_trans,total_units_department_610_1st_trans,total_units_department_612_1st_trans,total_units_department_615_1st_trans,total_units_department_710_1st_trans,total_units_department_800_1st_trans,week_recent_1_trans,week_counts_to_now_recent_one,label_1_recent_1_trans_also_1st,purchase_channel_1st_trans_recent_one,total_sales_recent_1_trans,total_units_recent_1_trans,total_units_department_minus_1_1st_trans_recent_one,total_units_department_108_1st_trans_recent_one,total_units_department_109_1st_trans_recent_one,total_units_department_110_1st_trans_recent_one,total_units_department_111_1st_trans_recent_one,total_units_department_114_1st_trans_recent_one,total_units_department_115_1st_trans_recent_one,total_units_department_120_1st_trans_recent_one,total_units_department_130_1st_trans_recent_one,total_units_department_140_1st_trans_recent_one,total_units_department_150_1st_trans_recent_one,total_units_department_160_1st_trans_recent_one,total_units_department_170_1st_trans_recent_one,total_units_department_210_1st_trans_recent_one,total_units_department_230_1st_trans_recent_one,total_units_department_250_1st_trans_recent_one,total_units_department_270_1st_trans_recent_one,total_units_department_310_1st_trans_recent_one,total_units_department_320_1st_trans_recent_one,total_units_department_330_1st_trans_recent_one,total_units_department_340_1st_trans_recent_one,total_units_department_350_1st_trans_recent_one,total_units_department_351_1st_trans_recent_one,total_units_department_352_1st_trans_recent_one,total_units_department_353_1st_trans_recent_one,total_units_department_354_1st_trans_recent_one,total_units_department_355_1st_trans_recent_one,total_units_department_360_1st_trans_recent_one,total_units_department_361_1st_trans_recent_one,total_units_department_362_1st_trans_recent_one,total_units_department_363_1st_trans_recent_one,total_units_department_364_1st_trans_recent_one,total_units_department_365_1st_trans_recent_one,total_units_department_366_1st_trans_recent_one,total_units_department_367_1st_trans_recent_one,total_units_department_370_1st_tra

In [91]:
df_2_2[pd.isnull(df_2_2['week_recent_5_trans'])]

,customer_id_hashed,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,total_units_department_minus_1_1st_trans,total_units_department_108_1st_trans,total_units_department_109_1st_trans,total_units_department_110_1st_trans,total_units_department_111_1st_trans,total_units_department_114_1st_trans,total_units_department_115_1st_trans,total_units_department_120_1st_trans,total_units_department_130_1st_trans,total_units_department_140_1st_trans,total_units_department_150_1st_trans,total_units_department_160_1st_trans,total_units_department_170_1st_trans,total_units_department_210_1st_trans,total_units_department_230_1st_trans,total_units_department_250_1st_trans,total_units_department_270_1st_trans,total_units_department_310_1st_trans,total_units_department_320_1st_trans,total_units_department_330_1st_trans,total_units_department_340_1st_trans,total_units_department_350_1st_trans,total_units_department_351_1st_trans,total_units_department_352_1st_trans,total_units_department_353_1st_trans,total_units_department_354_1st_trans,total_units_department_355_1st_trans,total_units_department_360_1st_trans,total_units_department_361_1st_trans,total_units_department_362_1st_trans,total_units_department_363_1st_trans,total_units_department_364_1st_trans,total_units_department_365_1st_trans,total_units_department_366_1st_trans,total_units_department_367_1st_trans,total_units_department_370_1st_trans,total_units_department_410_1st_trans,total_units_department_425_1st_trans,total_units_department_470_1st_trans,total_units_department_480_1st_trans,total_units_department_510_1st_trans,total_units_department_520_1st_trans,total_units_department_521_1st_trans,total_units_department_526_1st_trans,total_units_department_530_1st_trans,total_units_department_540_1st_trans,total_units_department_550_1st_trans,total_units_department_560_1st_trans,total_units_department_608_1st_trans,total_units_department_610_1st_trans,total_units_department_612_1st_trans,total_units_department_615_1st_trans,total_units_department_710_1st_trans,total_units_department_800_1st_trans,week_recent_1_trans,week_counts_to_now_recent_one,label_1_recent_1_trans_also_1st,purchase_channel_1st_trans_recent_one,total_sales_recent_1_trans,total_units_recent_1_trans,total_units_department_minus_1_1st_trans_recent_one,total_units_department_108_1st_trans_recent_one,total_units_department_109_1st_trans_recent_one,total_units_department_110_1st_trans_recent_one,total_units_department_111_1st_trans_recent_one,total_units_department_114_1st_trans_recent_one,total_units_department_115_1st_trans_recent_one,total_units_department_120_1st_trans_recent_one,total_units_department_130_1st_trans_recent_one,total_units_department_140_1st_trans_recent_one,total_units_department_150_1st_trans_recent_one,total_units_department_160_1st_trans_recent_one,total_units_department_170_1st_trans_recent_one,total_units_department_210_1st_trans_recent_one,total_units_department_230_1st_trans_recent_one,total_units_department_250_1st_trans_recent_one,total_units_department_270_1st_trans_recent_one,total_units_department_310_1st_trans_recent_one,total_units_department_320_1st_trans_recent_one,total_units_department_330_1st_trans_recent_one,total_units_department_340_1st_trans_recent_one,total_units_department_350_1st_trans_recent_one,total_units_department_351_1st_trans_recent_one,total_units_department_352_1st_trans_recent_one,total_units_department_353_1st_trans_recent_one,total_units_department_354_1st_trans_recent_one,total_units_department_355_1st_trans_recent_one,total_units_department_360_1st_trans_recent_one,total_units_department_361_1st_trans_recent_one,total_units_department_362_1st_trans_recent_one,total_units_department_363_1st_trans_recent_one,total_units_department_364_1st_trans_recent_one,total_units_department_365_1st_trans_recent_one,total_units_department_366_1st_trans_recent_one,total_units_department_367_1st_trans_recent_one,total_units_department_370_1st_tra

In [92]:
df_2_POS=pd.merge(df_2_1,df_2_2,on="customer_id_hashed",how="outer")
print(df_2_POS.shape,df_2_POS['customer_id_hashed'].nunique())
df_2_POS.head(2)

(577766, 420) 577766


,customer_id_hashed,week_diff,total_trans_since_registration,total_items,total_sales,trans_in_store,unique_stores,trans_online,department_minus_one_trans,department_108_trans,department_109_trans,department_110_trans,department_111_trans,department_114_trans,department_115_trans,department_120_trans,department_130_trans,department_140_trans,department_150_trans,department_160_trans,department_170_trans,department_210_trans,department_230_trans,department_250_trans,department_270_trans,department_310_trans,department_320_trans,department_330_trans,department_340_trans,department_350_trans,department_351_trans,department_352_trans,department_353_trans,department_354_trans,department_355_trans,department_360_trans,department_361_trans,department_362_trans,department_363_trans,department_364_trans,department_365_trans,department_366_trans,department_367_trans,department_370_trans,department_410_trans,department_425_trans,department_470_trans,department_480_trans,department_510_trans,department_520_trans,department_521_trans,department_526_trans,department_530_trans,department_540_trans,department_550_trans,department_560_trans,department_608_trans,department_610_trans,department_612_trans,department_615_trans,department_710_trans,department_800_trans,week_1st_trans,purchase_channel_1st_trans,total_sales_1st_trans,total_units_1st_trans,total_units_department_minus_1_1st_trans,total_units_department_108_1st_trans,total_units_department_109_1st_trans,total_units_department_110_1st_trans,total_units_department_111_1st_trans,total_units_department_114_1st_trans,total_units_department_115_1st_trans,total_units_department_120_1st_trans,total_units_department_130_1st_trans,total_units_department_140_1st_trans,total_units_department_150_1st_trans,total_units_department_160_1st_trans,total_units_department_170_1st_trans,total_units_department_210_1st_trans,total_units_department_230_1st_trans,total_units_department_250_1st_trans,total_units_department_270_1st_trans,total_units_department_310_1st_trans,total_units_department_320_1st_trans,total_units_department_330_1st_trans,total_units_department_340_1st_trans,total_units_department_350_1st_trans,total_units_department_351_1st_trans,total_units_department_352_1st_trans,total_units_department_353_1st_trans,total_units_department_354_1st_trans,total_units_department_355_1st_trans,total_units_department_360_1st_trans,total_units_department_361_1st_trans,total_units_department_362_1st_trans,total_units_department_363_1st_trans,total_units_department_364_1st_trans,total_units_department_365_1st_trans,total_units_department_366_1st_trans,total_units_department_367_1st_trans,total_units_department_370_1st_trans,total_units_department_410_1st_trans,total_units_department_425_1st_trans,total_units_department_470_1st_trans,total_units_department_480_1st_trans,total_units_department_510_1st_trans,total_units_department_520_1st_trans,total_units_department_521_1st_trans,total_units_department_526_1st_trans,total_units_department_530_1st_trans,total_units_department_540_1st_trans,total_units_department_550_1st_trans,total_units_department_560_1st_trans,total_units_department_608_1st_trans,total_units_department_610_1st_trans,total_units_department_612_1st_trans,total_units_department_615_1st_trans,total_units_department_710_1st_trans,total_units_department_800_1st_trans,week_recent_1_trans,week_counts_to_now_recent_one,label_1_recent_1_trans_also_1st,purchase_channel_1st_trans_recent_one,total_sales_recent_1_trans,total_units_recent_1_trans,total_units_department_minus_1_1st_trans_recent_one,total_units_department_108_1st_trans_recent_one,total_units_department_109_1st_trans_recent_one,total_units_department_110_1st_trans_recent_one,total_units_department_111_1st_trans_recent_one,total_units_department_114_1st_trans_recent_one,total_units_department_115_1st_trans_recent_one,total_units_department_120_1st_trans_recent_one,total_units_department_130_1st_trans_recent_one,total_units_department_140_1st_tran

In [93]:
df_3=pd.read_csv("./df_2_Exposure.csv")
print(df_3.shape,df_3['customer_id_hashed'].nunique())

(577766, 5) 577766


In [94]:
df_4=pd.read_csv("./df_3_Activity.csv")
print(df_4.shape,df_4['customer_id_hashed'].nunique())

(577766, 5) 577766


In [95]:
print(df_1.shape,df_1['customer_id_hashed'].nunique())
df_ivs=pd.merge(df_1,df_3,on="customer_id_hashed",how="outer")
print("merged_in_df_3_exposure",datetime.datetime.now())
print(df_ivs.shape,df_ivs['customer_id_hashed'].nunique())


df_ivs=pd.merge(df_ivs,df_4,on="customer_id_hashed",how="outer")
print("merged_in_df_4_activity",datetime.datetime.now())
print(df_ivs.shape,df_ivs['customer_id_hashed'].nunique())

df_ivs=pd.merge(df_ivs,df_2_POS,on="customer_id_hashed",how="outer")
print("merged_in_df_2_POS",datetime.datetime.now())
print(df_ivs.shape,df_ivs['customer_id_hashed'].nunique())

(577766, 11) 577766
merged_in_df_3_exposure 2020-04-29 23:26:11.614788
(577766, 15) 577766
merged_in_df_4_activity 2020-04-29 23:26:12.589680
(577766, 19) 577766
merged_in_df_2_POS 2020-04-29 23:26:33.582732
(577766, 438) 577766


# DVs

In [96]:
list_ids=df_ivs['customer_id_hashed'].tolist()


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
list_files_POS=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_files_POS=[x for x in list_files_POS if "dailysales" in x.lower()]
list_files_POS=[x for x in list_files_POS if x.split("s/MediaStorm_")[1][:10]>="2019-09-01"]
list_files_POS.sort()
list_files_POS=list_files_POS[:4]
list_files_POS

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-07/MediaStormDailySales20190910-120643-163.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-14/MediaStormDailySales20190917-110425-322.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-21/MediaStormDailySales20190924-112126-506.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-28/MediaStormDailySales20191001-111241-312.txt']

In [97]:
week_i=1
df_sale_dv=df_ivs[['customer_id_hashed']]
for file in list_files_POS:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df[df['item_transaction_amt']>10]
    
    df=df.rename(columns={"item_transaction_amt":"sales_week_"+str(week_i)})
    df_sale_dv=pd.merge(df_sale_dv,df,on="customer_id_hashed",how="left")
    week_i+=1
df_sale_dv=df_sale_dv.fillna(0)
df_sale_dv.head(2)

,customer_id_hashed,sales_week_1,sales_week_2,sales_week_3,sales_week_4
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,0.0,0.0,0.0,0.0
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,0.0,0.0,0.0,0.0


In [98]:
for col in df_sale_dv.columns.tolist():
    if "sales" in col:
        dv_col=col.replace("sales_","DV_")
        df_sale_dv[dv_col]=np.where(df_sale_dv[col]>0,1,0)
        
        print(dv_col,df_sale_dv[df_sale_dv[dv_col]>0].shape)

DV_week_1 (35196, 6)
DV_week_2 (32472, 7)
DV_week_3 (32957, 8)
DV_week_4 (41640, 9)


In [99]:
df_sale_dv['DV_cumulative_week_1']=df_sale_dv['DV_week_1']

df_sale_dv['DV_cumulative_week_2']=df_sale_dv['DV_week_1']+df_sale_dv['DV_week_2']
df_sale_dv['DV_cumulative_week_2']=np.where(df_sale_dv['DV_cumulative_week_2']>0,1,0)

df_sale_dv['DV_cumulative_week_3']=df_sale_dv['DV_week_1']+df_sale_dv['DV_week_2']+df_sale_dv['DV_week_3']
df_sale_dv['DV_cumulative_week_3']=np.where(df_sale_dv['DV_cumulative_week_3']>0,1,0)

df_sale_dv['DV_cumulative_week_4']=df_sale_dv['DV_week_1']+df_sale_dv['DV_week_2']+df_sale_dv['DV_week_3']+df_sale_dv['DV_week_4']
df_sale_dv['DV_cumulative_week_4']=np.where(df_sale_dv['DV_cumulative_week_4']>0,1,0)

In [100]:
for col in df_sale_dv.columns.tolist():
    if "DV_cumulative" in col:
        print(col,df_sale_dv[df_sale_dv[col]>0].shape)

DV_cumulative_week_1 (35196, 13)
DV_cumulative_week_2 (62403, 13)
DV_cumulative_week_3 (85902, 13)
DV_cumulative_week_4 (111909, 13)


In [101]:
df_ivs.shape

(577766, 438)

In [102]:
df_data_to_fit=pd.merge(df_sale_dv,df_ivs,on="customer_id_hashed",how="outer")
df_data_to_fit.shape

(577766, 450)

In [103]:
df_data_to_fit.to_csv("./df_data_up_to_2019-08-31_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [104]:
df_data_to_fit.head(1000).to_csv("./sample_df_data_up_to_2019-08-31_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [105]:
os.listdir("./")

['.ipynb_checkpoints',
 'extract_from_MySQL_to_pivot_table_Act.ipynb',
 'BL_data_sample_activities_JL_2020-04-01.xlsx',
 'extract_from_MySQL_to_pivot_table_Impr_Click.ipynb',
 'BL_data_sample_imprs_clicks_JL_2020-04-01.xlsx',
 'Output_each_piece.ipynb',
 'df_1_CRM.csv',
 'df_2_Exposure.csv',
 'df_3_Activity.csv',
 'aggregate_together.ipynb',
 'new_3_sample557K_pred_pos_table_2_2.csv',
 'sample557K_pred_pos_table_2_1.csv',
 'df_data_up_to_2019-08-31_JL_2020-04-27.csv',
 'sample_df_data_up_to_2019-08-31_JL_2020-04-27.csv',
 'US_zip_to_BL_nearest_store_20190901_JL_2020-04-29.csv',
 'new_sample557K_pred_pos_table_2_1.csv',
 'new_5_sample557K_pred_pos_table_2_2.csv',
 'df_data_up_to_2019-08-31_JL_2020-04-29.csv',
 'sample_df_data_up_to_2019-08-31_JL_2020-04-29.csv']

In [106]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/final_tables_to_use_in_model'